In [35]:
import websocket
import datetime
import hashlib
import base64
import hmac
import json
from urllib.parse import urlencode
import time
import ssl
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import _thread as thread
import os


STATUS_FIRST_FRAME = 0  # 第一帧的标识
STATUS_CONTINUE_FRAME = 1  # 中间帧标识
STATUS_LAST_FRAME = 2  # 最后一帧的标识


class Ws_Param(object):
    # 初始化
    def __init__(self, APPID, APIKey, APISecret, Text):
        self.APPID = APPID
        self.APIKey = APIKey
        self.APISecret = APISecret
        self.Text = Text

        # 公共参数(common)
        self.CommonArgs = {"app_id": self.APPID}
        # 业务参数(business)，更多个性化参数可在官网查看
        self.BusinessArgs = {"aue": "raw", "auf": "audio/L16;rate=16000", "vcn": "xiaoyan", "tte": "utf8","ent":"aisound"}
        self.Data = {"status": 2, "text": str(base64.b64encode(self.Text.encode('utf-8')), "UTF8")}
        #使用小语种须使用以下方式，此处的unicode指的是 utf16小端的编码方式，即"UTF-16LE"”
        #self.Data = {"status": 2, "text": str(base64.b64encode(self.Text.encode('utf-16')), "UTF8")}

    # 生成url
    def create_url(self):
        url = 'wss://tts-api.xfyun.cn/v2/tts'
        # 生成RFC1123格式的时间戳
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))

        # 拼接字符串
        signature_origin = "host: " + "ws-api.xfyun.cn" + "\n"
        signature_origin += "date: " + date + "\n"
        signature_origin += "GET " + "/v2/tts " + "HTTP/1.1"
        # 进行hmac-sha256进行加密
        signature_sha = hmac.new(self.APISecret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')

        authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
            self.APIKey, "hmac-sha256", "host date request-line", signature_sha)
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
        # 将请求的鉴权参数组合为字典
        v = {
            "authorization": authorization,
            "date": date,
            "host": "ws-api.xfyun.cn"
        }
        # 拼接鉴权参数，生成url
        url = url + '?' + urlencode(v)
        # print("date: ",date)
        # print("v: ",v)
        # 此处打印出建立连接时候的url,参考本demo的时候可取消上方打印的注释，比对相同参数时生成的url与自己代码生成的url是否一致
        # print('websocket url :', url)
        return url

def on_message(ws, message):
    try:
        message =json.loads(message)
        code = message["code"]
        sid = message["sid"]
        audio = message["data"]["audio"]
        audio = base64.b64decode(audio)
        status = message["data"]["status"]
        print(message)
        if status == 2:
            print("ws is closed")
            ws.close()
        if code != 0:
            errMsg = message["message"]
            print("sid:%s call error:%s code is:%s" % (sid, errMsg, code))
        else:

            with open('./demo.pcm', 'ab') as f:
                f.write(audio)

    except Exception as e:
        print("receive msg,but parse exception:", e)



# 收到websocket错误的处理
def on_error(ws, error):
    print("### error:", error)


# 收到websocket关闭的处理
def on_close(ws):
    print("### closed 合成成功，音频保存在./demo.pcm中###")


# 收到websocket连接建立的处理
def on_open(ws):
    def run(*args):
        d = {"common": wsParam.CommonArgs,
             "business": wsParam.BusinessArgs,
             "data": wsParam.Data,
             }
        d = json.dumps(d)
        print("------>开始发送文本数据")
        ws.send(d)
        if os.path.exists('./demo.pcm'):
            os.remove('./demo.pcm')

    thread.start_new_thread(run, ())


if __name__ == "__main__":
    # 测试时候在此处正确填写相关信息即可运行
    wsParam = Ws_Param(APPID='5e02cbbb', APIKey='5dd0f6f327116ad0faf0e55ec684b19e',
                       APISecret='b86188179ea7a4fdfab2ff7380e12c3e',
                       Text='The terrible earthquake and the following tsunami have caused so many deaths. ')
    websocket.enableTrace(False)
    wsUrl = wsParam.create_url()
    ws = websocket.WebSocketApp(wsUrl, on_message=on_message, on_error=on_error, on_close=on_close)
    ws.on_open = on_open
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})



------>开始发送文本数据
{'code': 0, 'message': 'success', 'sid': 'tts000649c6@dx16f3b2a836c7a1c902', 'data': {'status': 1, 'ced': '76', 'audio': '...'}}
{'code': 0, 'message': 'success', 'sid': 'tts000649c6@dx16f3b2a836c7a1c902', 'data': {'audio': '...', 'status': 1, 'ced': '76'}}
{'code': 0, 'message': 'success', 'sid': 'tts000649c6@dx16f3b2a836c7a1c902', 'data': {'status': 1, 'ced': '76', 'audio': '...'}}
{'code': 0, 'message': 'success', 'sid': 'tts000649c6@dx16f3b2a836c7a1c902', 'data': {'audio': 'Nv9A/9L/g//3/87/UP/t/2L/Rv+V/xT/+P6k/xf//v/b/+z/SwCO/xUAMP8EAPT/hv9TACn/QQDrAGb/KgEdANv/NQE+/20BsgAQAKgBn/8OARgBhwC7AbwAhwCvAM7/WgBgAHkAeQA2ALIAFQBeAJr/0P8VABgAegB8/7T/iP8s/4D/Kf/r/sH/7v43AFr/tv8/AIf+IAB9/pj/1f97/ggATP6i/2cAe/7ZAIX/iP/OAJH+TAEWAOD/iQEU/0IBvABWAJkBWwC/ALQAbQB3AIYAxAAoAdkAhwB+AGUAqf/5/9X/Bf9oACL/eQDl/wgAlQDU/lMAff6G/9f/TP4LAGD+jf+5AC7+8gBq/xL/7QAP/jAByP9p/1sBkv4MAV0A8v9/Ad//WABQAGr/4wC8/0MA6gAu/7UAg/89/28AIf86ARYAbADBAOP+bwC9/gAAPwAS/6AAjf45ANYAiP5MAWH/wf8GAX/+rgG0/7v/RAF1/voAWwASA